# Projeto 1 - Ciência dos Dados

Nome: João Pedro Pereira Silva Marques de Oliveira

Nome: Rafael Alves Madarás

___
Carregando algumas bibliotecas e o arquivo de base de dados:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
from emoji import UNICODE_EMOJI
from math import *

In [2]:
filename = 'coca cola.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com o projeto!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo coca cola.xlsx, tudo certo para prosseguir com o projeto!


# Base de dados

Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
treino = pd.read_excel(filename)
treino.head(3)

,Treinamento,Relevancia
0,esse tic tac de coca cola é bom dms pqp,1
1,povo aq de casa tão tudo viciados em coca cola...,1
2,@mariano_mem acho que coca cola,0


In [4]:
teste = pd.read_excel(filename, sheet_name = 'Teste')
teste.drop("Unnamed: 2",inplace=True,axis=1)
teste.head(3)

,Teste,Relevancia
0,@steffauxx la no jk com uma body piercing perf...,0
1,do nada uma puta vontade de tomar uma coca-col...,1
2,saudades de quando eu não ligava pra esse tipo...,1


___
# Classificador automático de sentimento


Para a realização desse projeto, tivemos o objetivo de escolher um produto e analisar diversos tweets sobre o mesmo, classificando-os como "relevantes" ou "irrelevantes" e montar um classificador automático de tweets. Para tal, o produto escolhido foi o mais famoso refrigerante do mundo: a Coca-Cola. Sabendo da grande fama da marca, era óbvio que haveria diversos tweets contendo seu nome. Portanto, foram utilizados os seguintes critérios para a realização da classificação dos tweets: 

- Relevantes(1): Tweets que continham o nome da marca sendo utilizado com referência direta ao próprio produto, ou expressões de cunho positivo ou negativo que agregassem valor à marca.


- Irrelevantes(0): Tweets que continham o nome da marca, mas não faziam referência direta ao próprio produto, nem agregavam valor à ela, ou que continham o nome inserido numa frase sem contexto.

___
# Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

## Funções utilizadas

In [5]:
# função que define emojis como emoji
def emoji(s):
    return s in UNICODE_EMOJI

# função que adiciona espaço entre emojis/ emoji e palavra
def adiciona_espaco_emoji(text):
    return ''.join(' ' + char + ' 'if emoji(char) else char for char in text).strip()

# função que filtra o texto e retira as pontuações indesejadas
def tirar_pont(text):
    punctuation = '[!-.:?;"\n"()''"",_%$੭𓂃\—﹢ׄ›ー|…/»–]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

# função que filtra o texto e retira os links indesejados
def tirar_link(text):
    link = r'http[^\s]*'    
    pattern = re.compile(link)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

# função que filtra o texto e retira os espaços indesejados
def tirar_esp(text):
    esp2 = '  '
    pattern = re.compile(esp2)
    text_subbed = re.sub(esp2, ' ', text)
    return text_subbed

# função com todos os filtors anteriores
def filtros(texto):
    parte_1 = texto.lower()
    parte_2 = tirar_link(parte_1)
    parte_3 = tirar_pont(parte_2)
    parte_4 = tirar_esp(parte_3)
    return adiciona_espaco_emoji(parte_4)

# função que transforma texto em uma unica string
def transforma_str(texto):
    string = str()
    for i in texto:
        string +=' '+i
    return string

## Obtenção das variáveis necessárias para criar o classificador

### Variáveis gerais do classificador

In [6]:
# transforma a base de dados (treino) em variável categórica e define os nomes
treino["Relevancia"] = treino["Relevancia"].astype('category')
treino["Relevancia"].cat.categories = ["Irrelevante","Relevante"]

# transforma a base de dados em uma string
treino_str = transforma_str(treino['Treinamento'])

# implementação dos filtros na string da base de dados
texto_espaco_emoji = filtros(treino_str)

# variáveis necessárias para o smoothing
# Numero de palavras potencialmente existentes
V = 10e5

# "bonus" na contagem da palavra
alpha = 1

### Variáveis de tweets relevantes do classificador

In [7]:
# serie do Pandas dos tweets relevantes
relevante = treino['Relevancia'] == "Relevante"

# tabela com os tweets relevantes
tweets_relevantes = treino.loc[relevante,:]

# transforma em uma unica string os tweets relevantes
treino_str_relevante = transforma_str(tweets_relevantes["Treinamento"])

# filtra string dos tweets relevantes
str_clean_tweets_r = filtros(treino_str_relevante)

# cria uma lista com cada termo da string dos tweets relevantes
palavras_r = str_clean_tweets_r.split()

# transforma tweets relevantes em serie do Pandas
treinamento_r = pd.Series(palavras_r)


# cria lista com cada tweet relevante
lista_relevante = []
for i in tweets_relevantes["Treinamento"]:
    lista_relevante.append(filtros(i))
    

#Frequencia absoluta das palavras relevantes e sua soma
FR_r = treinamento_r.value_counts()
soma_r = sum(FR_r)


# P(palavra|relevante)
prob_palavra_dado_r = (FR_r + alpha)/(soma_r + V*alpha)


# P(tweet|relevante) 
# em uma lista (ordem da lista = ordem dos tweets)
lista_r = []
for tweet in lista_relevante:
    prob_f_r = log(treino["Relevancia"].value_counts(True)[1])
    for palavra in tweet.split():
        if palavra in prob_palavra_dado_r:
            prob_f_r += log(prob_palavra_dado_r[palavra])
        else:
            prob_f_r += log(alpha/soma_r + V*alpha)         
    lista_r.append(prob_f_r)

# Dicionário[tweet_relevante] = P(tweet|relevante)    
dic_relevante = {} 
for i in range(len(lista_relevante)):
    dic_relevante[lista_relevante[i]] = lista_r[i]

### Variáveis de tweets irrelevantes do classificador

In [8]:
# serie do Pandas dos tweets irrelevantes
irrelevante = treino["Relevancia"] == "Irrelevante"

# tabela com tweets irrelevantes
tweets_irrelevantes = treino.loc[irrelevante,:]

# transforma em uma unica string os tweets irrelevantes
treino_str_irrelevante = transforma_str(tweets_irrelevantes["Treinamento"])

# filtra string com os tweets irrelevantes
str_clean_tweets_i = filtros(treino_str_irrelevante)

# cria uma lista com cada termo da string dos tweets irrelevante
palavras_i = str_clean_tweets_i.split()

# transforma tweets irrelevantes em serie do Pandas
treinamento_i = pd.Series(palavras_i)

# cria lista com cada tweet irrelevante
lista_irrelevante = []
for i in tweets_irrelevantes["Treinamento"]:
    lista_irrelevante.append(filtros(i))

#Frequencia absoluta das palavras irrelevantes e sua soma
FR_i = treinamento_i.value_counts()
soma_i = sum(FR_i)

# P(palavra|irrelevante)
prob_palavra_dado_i = (FR_i + alpha)/(soma_i + V*alpha)

# P(tweet|irrelevante)
# em uma lista (ordem da lista = ordem dos tweets)
lista_i = []
for tweet in lista_irrelevante:
    prob_f_i = log(treino["Relevancia"].value_counts(True)[0])
    for palavra in tweet.split():
        if palavra in prob_palavra_dado_i:
            prob_f_i += log(prob_palavra_dado_i[palavra])
        else:
            prob_f_i += log(alpha/soma_i + V*alpha)       
    lista_i.append(prob_f_i)

# Dicionário[tweet_irrelevante] = P(tweet|irrelevante)
dic_irrelevante = {}    
for i in range(len(lista_irrelevante)):
    dic_irrelevante[lista_irrelevante[i]] = lista_i[i]

## Classificador

In [9]:
# cria dicionário no formato: dic_resultado[tweet] = relevância_do_tweet
dic_resultado = {}
for tweet_r, tweet_r_prob in dic_relevante.items(): 
    for tweet_i,tweet_i_prob in dic_irrelevante.items():
        
        if tweet_r == tweet_i: 
            if tweet_r_prob > tweet_i_prob:
                dic_resultado[tweet_r] = "relevante"   
            else: 
                dic_resultado[tweet_i] = "irrelevante" 
                
        if tweet_r != tweet_i and tweet_r not in dic_resultado:
            dic_resultado[tweet_r] = "relevante"   
            
        if tweet_r != tweet_i and tweet_i not in dic_resultado:
            dic_resultado[tweet_i] = "irrelevante"

# cria lista com todos os tweets filtrados 
lista_tweets_total = []
for i in treino["Treinamento"]:
    lista_tweets_total.append(filtros(i))

# transforma lista_tweets_total em lista do pandas e adiciona ao DataFrame "treino"
Treinamento_filtrado_NB = pd.Series(lista_tweets_total)
treino['Treinamento_filtrado_NB'] = Treinamento_filtrado_NB
        
# dicionário final no formato: dic_final[tweet] = relevancia 
# esse dicionário foi utilizado para deixar o dic_resultado na ordem da lista_tweets_total
dic_final = {}
for tweet in lista_tweets_total:
    for tweet_resultado, relevancia in dic_resultado.items():
        if tweet_resultado == tweet:
            dic_final[tweet] = relevancia 

# lista com a relevancia de cada tweet na ordem do respectivo tweets na lista_tweets_total
lista_relevancia = []
for relevancia in dic_final.values():
    lista_relevancia.append(relevancia)

# adiciona ao DataFrame "treino" a tabela "Relevancia_NB"
Relevancia_NB = pd.Series(lista_relevancia)
treino["Relevancia_NB"] = Relevancia_NB

Tabela de comparação entre a porcentagem de relevância dos tweets determinados previamente (pelo excel) e porcentagem de relevância dos tweets determinados pelo nosso classificador, com base na planilha de treinamento.

In [10]:
tabela_comparacao_treino = pd.crosstab(treino["Relevancia"],treino["Relevancia_NB"],normalize=True)
tabela_comparacao_treino

Relevancia_NB,irrelevante,relevante
Relevancia,,
Irrelevante,0.367893,0.140468
Relevante,0.143813,0.347826


### Probabilidades por categoria  (Treinamento)
#### Probabilidade de verdadeiros positivos

In [11]:
P_VP = ((tabela_comparacao_treino.iloc[1,1])*100).round(5)
print("P(verdadeiros_positivos) = ",P_VP,"%")

P(verdadeiros_positivos) =  34.78261 %


#### Probabilidade de verdadeiros negativos:

In [12]:
P_VN = ((tabela_comparacao_treino.iloc[1,0])*100).round(5)
print("P(verdadeiros_negativos) = ",P_VN,"%")

P(verdadeiros_negativos) =  14.38127 %


#### Probabilidade de falsos positivos:

In [13]:
P_FP = ((tabela_comparacao_treino.iloc[0,0])*100).round(5)
print("P(falsos_positivoss) = ",P_FP,"%")

P(falsos_positivoss) =  36.7893 %


#### Probabilidade de falsos negativos:

In [14]:
P_FN = ((tabela_comparacao_treino.iloc[0,1])*100).round(5)
print("P(falsos_negativos) = ",P_FN,"%")

P(falsos_negativos) =  14.04682 %


___
# Verificando a performance do Classificador

Agora você deve testear o seu classificador com a base de testes.

In [15]:
# transforma a base de dados (teste) em variável categórica e define os nomes
teste["Relevancia"] = teste["Relevancia"].astype('category')
teste["Relevancia"].cat.categories = ["Irrelevante","Relevante"]

# cria lista com todos os tweets filtrados
lista_tweets_teste = []
for i in teste["Teste"]:
    lista_tweets_teste.append(filtros(i))

# transforma lista_tweets_teste em lista do pandas e adiciona ao DataFrame "teste"
Teste_filtrado_NB_teste = pd.Series(lista_tweets_teste)
teste['Teste_filtrado'] = Teste_filtrado_NB_teste
        
# dicionário final no formato: dic_final[tweet] = relevancia 
dic_final_teste = {}
for tweet in lista_tweets_teste:
    for tweet_resultado,relevancia in dic_resultado.items():
        if tweet_resultado == tweet:
            dic_final_teste[tweet] = relevancia 
            
# lista com a relevancia de cada tweet na ordem do respectivo tweets na lista_tweets_total
lista_relevancia_teste = []
for relevancia in dic_final_teste.values():
    lista_relevancia_teste.append(relevancia)
    
# adiciona ao DataFrame "treino" a tabela "Relevancia_NB"
Relevancia_NB_teste = pd.Series(lista_relevancia)
teste["Relevancia_NB"] = Relevancia_NB_teste

In [16]:
tabela_comparacao = pd.crosstab(teste["Relevancia"],teste["Relevancia_NB"],normalize=True)
tabela_comparacao

Relevancia_NB,irrelevante,relevante
Relevancia,,
Irrelevante,0.240,0.190
Relevante,0.295,0.275


### Probabilidades por categoria  (teste)
#### Probabilidade de verdadeiros positivos

In [17]:
P_VP = ((tabela_comparacao.iloc[1,1])*100).round(5)
print("P(verdadeiros_positivos) = ",P_VP,"%")

P(verdadeiros_positivos) =  27.5 %


#### Probabilidade de verdadeiros negativos:

In [18]:
P_VN = ((tabela_comparacao.iloc[1,0])*100).round(5)
print("P(verdadeiros_negativos) = ",P_VN,"%")

P(verdadeiros_negativos) =  29.5 %


#### Probabilidade de falsos positivos:

In [19]:
P_FP = ((tabela_comparacao.iloc[0,0])*100).round(5)
print("P(falsos_positivoss) = ",P_FP,"%")

P(falsos_positivoss) =  24.0 %


#### Probabilidade de falsos negativos:

In [20]:
P_FN = ((tabela_comparacao.iloc[0,1])*100).round(5)
print("P(falsos_negativos) = ",P_FN,"%")

P(falsos_negativos) =  19.0 %


### Acurácia

In [21]:
acuracia = (tabela_comparacao.iloc[0,0]+tabela_comparacao.iloc[1,1])*100
print("A acurácia do nosso classificador é de ", acuracia,"%.")

A acurácia do nosso classificador é de  51.5 %.


___
### Concluindo

Utilizando a planilha de treino ("Treinamento") e as noções de probabilidade aprendidas em aula, montamos nosso classificador Naive-Bayes e o testamos. Para isso, utilizamos uma outra planilha, dessa vez com tweets diferentes da anterior: a planilha "Teste". Ao finalizar os testes, obtivemos as probabilidades indicadas acima. Com elas, é possível observar que o classficador não é muito eficiente e funciona um pouco melhor para os tweets relevantes do que para os irrelevantes. Também obtivemos a acurácia total do classificador, que é por volta de 50%.

Dado que só trabalhamos com duas categorias ("irrelevante" e "relevante"), é impossivel afirmar com certeza o destino das mensagens que continham dupla negação ou sarcasmo que passaram pelo classificador. Para começar, é importante frisar que qualquer tweet que agregava valor ao produto foi classificado por nós como relevante. Dado que as mensagens sarcásticas da base de treinamento em sua maioria agragavam valor ao nome da marca, muito provavelmente elas foram classificadas como relevantes, induzindo o classificador a colocar mais mensagens como ela nessa categoria, independente da intenção do tweet. Para os tweets com dupla negação, seus destinos são bem mais incertos, pois, como existe a ocorrência de palavras de sentido negativo mais de uma vez, o classificador acaba se confundindo com o real sentido do tweet, e assim classifica-o de forma incorreta.

Mesmo com um classificador que tem um percentual de eficiêcia que se assemelha ao giro de uma moeda, ele ainda pode ter muita utilidade e ser muito mais eficiente. Com uma base de dados maior e mais precisa, as probabilidades dele classificar um tweet como relevante ou irrelevante, seriam muito mais próximas da realidade. Além disso, com mais tempo de desenvolvimento, seria possível tornar o código mais complexo ao adicionar novas funcionalidades. Dentre as quais podemos citar a importância na ordem das palavras e o fato das palavras não ocorrerem de forma independente no tweet.

Mesmo que seja um classificador simples, o Naive-Bayes tem várias funcionalidades e pode ser aplicado em diversas ocasiões. Uma delas, por exemplo, é a organização de uma caixa de entrada e emails, onde, através do classificador, seria possível categorizar os emails em spam e não spam. Outra aplicação possível para o Naive-Bayes é a identificação do assunto de uma notícia.

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**